In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

eval_set = load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")['eval']


# model_tag = "pythia28_hh_selected_clean_subset"  

# model_name_or_path = f"/mnt/data1/jinlong/compare_model_preferences/{model_tag}_model"

# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# def generate(instruction):
#     inputs = tokenizer(instruction, return_tensors="pt")    
#     outputs = model.generate(**inputs, max_length=1024)  
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# for example in tqdm(eval_set, desc="Generating AlpacaEval2's Repsonses"):
#     example['output'] = generate(example['instruction'])
#     example['generator'] = model_tag



/home/jlpang/alpaca_eval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "/mnt/data1/jinlong/DPO-noisy-outputs/llama-3-8b-kto"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)  # You may want to use bfloat16 and/or move to GPU here

messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

/home/jlpang/alpaca_eval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.88s/it]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a friendly chatbot who always responds in the style of a pirate<|eot_id|><|start_header_id|>user<|end_header_id|>

How many helicopters can a human eat in one sitting?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [9]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

# 加载评估数据集
eval_set = load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")['eval']

# 模型路径和标签
model_tag = "pythia28_hh_selected_clean_subset"
model_name_or_path = f"/mnt/data1/jinlong/compare_model_preferences/{model_tag}_model"

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# 设置批处理大小
batch_size = 16

def generate_batch(instructions):
    # 将一批 instruction 编码成模型能理解的格式
    inputs = tokenizer(instructions, padding=True, truncation=True, return_tensors="pt")
    
    # 将所有输入传递给模型并生成输出
    outputs = model.generate(**inputs, max_length=1024)
    
    # 解码生成的输出并返回
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# 按批次生成输出
generated_outputs = []
for i in tqdm(range(0, len(eval_set), batch_size), desc="Generating AlpacaEval2's Responses"):
    batch = eval_set[i:i+batch_size]  # 获取当前批次的数据
    instructions = [example['instruction'] for example in batch]  # 提取 instruction 列表
    batch_outputs = generate_batch(instructions)  # 获取当前批次的生成输出
    
    # 将输出添加到结果列表
    for example, output in zip(batch, batch_outputs):
        example['output'] = output
        example['generator'] = model_tag
        generated_outputs.append(example)


Generating AlpacaEval2's Responses:   0%|          | 0/51 [00:00<?, ?it/s]


TypeError: string indices must be integers

In [4]:
from datasets import load_dataset



# eval_set = load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")['eval']
reference_outputs = load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval_gpt4_baseline")['eval']

In [7]:
reference_outputs[0]

{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
 'output': 'Several famous actors started their careers on Broadway before making it big in film and television. Here are a few notable examples:\n\n1. Sarah Jessica Parker - Before she was Carrie Bradshaw on "Sex and the City," Sarah Jessica Parker was a Broadway star, having appeared in productions like "Annie" as a child.\n\n2. Meryl Streep - Meryl Streep\'s early career included Broadway productions such as "Trelawny of the \'Wells\'" and "A Memory of Two Mondays / 27 Wagons Full of Cotton."\n\n3. Hugh Jackman - Hugh Jackman won a Tony Award for his role in "The Boy from Oz" and has been known for his stage work as well as his film career.\n\n4. Sutton Foster - Known for her television role in "Younger," Sutton Foster is also a Broadway legend with leading roles in shows like "Thoroughly Modern Millie" and "Anything Goes."\n\n5. Kristen Bell - Before she was the voice of Anna in "Fro

In [2]:
eval_set

Dataset({
    features: ['instruction', 'output', 'generator', 'dataset'],
    num_rows: 805
})

In [5]:
import pandas as pd

# df = pd.read_json('./results/llama-3-1b-dpo/model_outputs.json')

# loss_type = "dpo-new"
loss_type = "ours1-1"
leaderboard = pd.read_csv(f"model_outputs/llama-3-8b-{loss_type}-merged/alpaca_eval_gpt4/leaderboard.csv")
leaderboard

,Unnamed: 0,win_rate,standard_error,mode,avg_length,n_wins,n_wins_base,n_draws,n_total,discrete_win_rate,length_controlled_winrate,lc_standard_error
0,llama-3-8b-ours1-1-merged,42.105263,11.63728,community,3441,8,11,0,19,42.105263,39.924597,2.802031


In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="model_outputs/llama-3-8b-ours4-1-new1/model_outputs_full.json")['train']

In [8]:
lens = []

count = 0
for output in dataset['output']:
    if output == '':
        count+=1
    lens.append(len(output))



In [1]:
from datasets import load_dataset

eval_set = load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")['eval']


In [1]:
from transformers import AutoModelForCausalLM

loss_types = ["SimPO", "KTO", 'IPO'] # 'CPO',  'DPO'

for loss_type in loss_types:

    model_name_or_path = f"princeton-nlp/Llama-3-Base-8B-SFT-{loss_type}"
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path)


/home/jlpang/alpaca_eval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.59it/s]


In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer

loss_types = ["SimPO", "KTO", 'IPO', 'CPO',  'DPO'] # 

for loss_type in loss_types:

    model_name_or_path = f"princeton-nlp/Mistral-7B-Base-SFT-{loss_type}"
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
    tokenizer =AutoTokenizer.from_pretrained(model_name_or_path)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.29it/s]


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("princeton-nlp/Llama-3-Base-8B-SFT-SimPO")
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/Llama-3-Base-8B-SFT-CPO")

model.push_to_hub("jlpang888/Llama-3-Base-8B-SFT-SimPO")
tokenizer.push_to_hub("jlpang888/Llama-3-Base-8B-SFT-SimPO")


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.64it/s]
model-00002-of-00007.safetensors: 100%|██████████| 4.83G/4.83G [02:33<00:00, 31.5MB/s]
tokenizer.json: 100%|██████████| 17.2M/17.2M [00:00<00:00, 30.8MB/s]


CommitInfo(commit_url='https://huggingface.co/jlpang888/Llama-3-Base-8B-SFT-SimPO/commit/b666b67c788f90a10029cb8ff5af12a78d3faf89', commit_message='Upload tokenizer', commit_description='', oid='b666b67c788f90a10029cb8ff5af12a78d3faf89', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jlpang888/Llama-3-Base-8B-SFT-SimPO', endpoint='https://huggingface.co', repo_type='model', repo_id='jlpang888/Llama-3-Base-8B-SFT-SimPO'), pr_revision=None, pr_num=None)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name="princeton-nlp/Llama-3-Base-8B-SFT-SLiC-HF"
model_name = "princeton-nlp/Mistral-7B-Base-SFT-SLiC-HF"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

KeyboardInterrupt: 

In [1]:
import json
from transformers import AutoTokenizer
import numpy as np

# Llama-3-Base-8B-SFT-SimPO
model_name_or_path = "Llama-3-Base-8B-SFT-SimPO"
json_path = "model_outputs_cl/Llama-3-Base-8B-SFT-SimPO/model_outputs_full.json"

# 修改为你的 model 路径
# model_name_or_path = "/mnt/data1/jinlong/CL_DPO_outputs/llama-3-8b-dpo-sorted-llama-full"
# json_path = "./model_outputs_cl/llama-3-8b-dpo-sorted-llama-full-original/model_outputs_full.json"

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# 加载生成结果
with open(json_path, 'r') as f:
    data = json.load(f)

# 获取 response 列表
responses = data['output']

# 字符长度统计
char_lengths = [len(r) for r in responses]

# token 数统计
token_lengths = [len(tokenizer.encode(r, add_special_tokens=False)) for r in responses]

print(f"Number of responses: {len(responses)}")
print(f"Avg response length (chars): {np.mean(char_lengths):.2f}")
print(f"Avg response length (tokens): {np.mean(token_lengths):.2f}")


/home/jlpang/alpaca_eval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of responses: 805
Avg response length (chars): 7761.79
Avg response length (tokens): 1620.59


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 模型名称，例如 llama 或 mistral
model_name = "jlpang888/llama-3-8b-ultrafeedback-sft-chosen"

# 本地保存路径（不存在会自动创建）
save_directory = "/mnt/data1/jinlong/CL_DPO_outputs/llama-3-8b-base-sft-chosen"

# 加载模型和 tokenizer（会从 Hugging Face 下载）
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 保存到本地路径
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
